![Header image](resources/image.jpg)

Exciting times! 

You've been running our very successful gadget webshop 'DataGadgets' for a few years and have recently expanded into new territories. While you've been focussed on the US market for the first five years of our existence, you now are shipping our cool data gadgets to the UK and Europe, too! But now our in-house built reporting has broken! Transactions don't only come in USD, but you're also receiving transactions in EUR and GPB. 

To better understand the volume of transactions being made, you should convert the non-USD transactions to USD and sum up the total. To do this, however, you'll need to use the proper exchange rates. 

In this project, you'll start with a CSV file containing all e-commerce transactions made on January 21st, but in their original currencies. Your job is to calculate the total sum in USD of all transactions so you know how much USD you sold on January 21st. To do this, you'll need to convert any non-USD transactions to USD using the exchange rate from January 21st, 2024. 

To get the exchange rates from January 21st, 2024, you'll rely on [VAT Comply rates API's](https://www.vatcomply.com/documentation#rates) public and free currency exchange API. You'll also use `pandas` to load the CSV file into a DataFrame and the `requests` package to make interacting with the API easier. 

You need to update the `orders` DataFrame so the final version has two new columns: `exchange_rate` and `amount_usd`. The final version should look as follows:

| `amount` | `currency` | `exchange_rate` | `amount_usd` |
|-|-|-|-|
| 43.75 | EUR | ... | ... |
| 385.5 | GBP | ... | ... |
| 495.5 | GBP | ... | ... |
| 117.99 | GBP | ... | ... |
| 624 | USD | ... | ... |

In [55]:
# Import required packages/libraries
import pandas as pd
import requests
import json

# Read the CSV file into a DataFrame
orders = pd.read_csv('data/orders-2024-01-21.csv')
orders.head()

,amount,currency
0,43.75,EUR
1,385.50,GBP
2,495.50,GBP
3,117.99,GBP
4,624.00,USD


In [56]:
# Define a function to get the exchange rate
def get_exchange_rate(json_data: dict, key: str) -> float:
    """Return the exchange rate by accessing the rate from a dictionary"""
    return json_data['rates'][key]
    
# Define a function to get the amount in USD
def calc_usd(amount: float, exchange_rate: float) -> float:
    """Converts a currency to USD based on a given exchange rate"""
    return amount * exchange_rate


# Define parameters
params = {'base':'USD', 'date':'2024-01-21'}

# Query the url
response = requests.get('https://api.vatcomply.com/rates', params=params)

# Transform the response text to a dictionary
rates = json.loads(response.text)

# Create empty lists to store our new columns
exchange_rate = []
amount_usd = []

# Apply the functions to the df and append to the lists
for row in orders.itertuples(index=False):
    exchange_rate.append(get_exchange_rate(rates, row.currency))

orders['exchange_rate'] = exchange_rate

for row in orders.itertuples(index=False):
    amount_usd.append(calc_usd(row.amount, row.exchange_rate))

orders['amount_usd'] = amount_usd

# Calculate total sales for the day
total_usd_sales = orders['amount_usd'].sum() 

# Preview the new df
orders.head()

,amount,currency,exchange_rate,amount_usd
0,43.75,EUR,0.918527,40.185542
1,385.50,GBP,0.788326,303.899490
2,495.50,GBP,0.788326,390.615298
3,117.99,GBP,0.788326,93.014529
4,624.00,USD,1.000000,624.000000


In [57]:
print(total_usd_sales)

326864.39599246805
